In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.util import ngrams

In [2]:
df = pd.read_csv('data/clean_Wpolarity.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.replace(to_replace=r"\[|\]|\' ", value='', inplace=True, regex=True)
df.head()

,id,keyword,location,text,target,polar
0,1,NaN,NaN,our deeds are the reason of this may allah fo...,1,0.2732
1,4,NaN,NaN,forest fire near la ronge sask canada,1,-0.3400
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,-0.2960
3,6,NaN,NaN,people receive evacuation orders in california,1,0.0000
4,7,NaN,NaN,just got sent this photo from ruby as smoke f...,1,0.0000


In [3]:
freqs = {}
for i in df['text']:
    tokens = nltk.tokenize.word_tokenize(i, language="english")
    for n in range(1, 3):
        for j in ngrams(tokens, n):
            j = str(j)
            freqs.setdefault(j, 0)
            freqs[j]+=1
freqs

{"('our',)": 99,
 "('deeds',)": 2,
 "('are',)": 401,
 "('the',)": 3261,
 "('reason',)": 20,
 "('of',)": 1823,
 "('this',)": 477,
 "('may',)": 88,
 "('allah',)": 4,
 "('forgive',)": 2,
 "('us',)": 163,
 "('all',)": 257,
 "('our', 'deeds')": 1,
 "('deeds', 'are')": 1,
 "('are', 'the')": 15,
 "('the', 'reason')": 2,
 "('reason', 'of')": 2,
 "('of', 'this')": 9,
 "('this', 'may')": 1,
 "('may', 'allah')": 3,
 "('allah', 'forgive')": 1,
 "('forgive', 'us')": 1,
 "('us', 'all')": 5,
 "('forest',)": 63,
 "('fire',)": 247,
 "('near',)": 54,
 "('la',)": 25,
 "('ronge',)": 1,
 "('sask',)": 1,
 "('canada',)": 9,
 "('forest', 'fire')": 8,
 "('fire', 'near')": 3,
 "('near', 'la')": 1,
 "('la', 'ronge')": 1,
 "('ronge', 'sask')": 1,
 "('sask', 'canada')": 1,
 "('residents',)": 8,
 "('asked',)": 9,
 "('to',)": 1945,
 "('shelter',)": 6,
 "('in',)": 1976,
 "('place',)": 26,
 "('being',)": 90,
 "('notified',)": 1,
 "('by',)": 519,
 "('officers',)": 8,
 "('no',)": 254,
 "('other',)": 36,
 "('evacuation',

In [4]:
indexer = {}
index = 0
for i in df['text']:
    tokens = nltk.tokenize.word_tokenize(i, language="english")
    for n in range(1, 3):
        for j in ngrams(tokens, n):
            j = str(j)
            if j not in indexer.keys() and freqs[j] >= 10:
                indexer[j] = index
                index+=1
indexer

{"('our',)": 0,
 "('are',)": 1,
 "('the',)": 2,
 "('reason',)": 3,
 "('of',)": 4,
 "('this',)": 5,
 "('may',)": 6,
 "('us',)": 7,
 "('all',)": 8,
 "('are', 'the')": 9,
 "('forest',)": 10,
 "('fire',)": 11,
 "('near',)": 12,
 "('la',)": 13,
 "('to',)": 14,
 "('in',)": 15,
 "('place',)": 16,
 "('being',)": 17,
 "('by',)": 18,
 "('no',)": 19,
 "('other',)": 20,
 "('evacuation',)": 21,
 "('or',)": 22,
 "('orders',)": 23,
 "('expected',)": 24,
 "('people',)": 25,
 "('california',)": 26,
 "('in', 'california')": 27,
 "('just',)": 28,
 "('got',)": 29,
 "('sent',)": 30,
 "('photo',)": 31,
 "('from',)": 32,
 "('as',)": 33,
 "('smoke',)": 34,
 "('into',)": 35,
 "('a',)": 36,
 "('school',)": 37,
 "('just', 'got')": 38,
 "('into', 'a')": 39,
 "('update',)": 40,
 "('closed',)": 41,
 "('both',)": 42,
 "('due',)": 43,
 "('lake',)": 44,
 "('county',)": 45,
 "('due', 'to')": 46,
 "('heavy',)": 47,
 "('rain',)": 48,
 "('causes',)": 49,
 "('flash',)": 50,
 "('flooding',)": 51,
 "('colorado',)": 52,
 "('i

In [28]:
len(indexer)*len(df)*4/10e9

0.0071927624

In [29]:
n_gram_onehot = np.zeros(shape=(len(df), len(indexer)+1), dtype=np.float32)
n_gram_onehot.shape

(7613, 2363)

In [30]:
import json
with open('hdt5indexes.sav', 'w') as f:
    f.write(json.dumps(indexer))

In [31]:
missed = 0
for i, e in enumerate(df['text']):
    tokens = nltk.tokenize.word_tokenize(e, language="english")
    matched = False
    for j in ngrams(tokens, n):
        j = str(j)
        if j in indexer.keys():
            n_gram_onehot[i][indexer[j]] = 1.0
            matched = True
    n_gram_onehot[i][len(indexer)] = df['polar'][i]
    if not matched:
        missed +=1
print(missed)

2352


In [32]:
labelsOneHot = np.zeros((n_gram_onehot.shape[0], 2))
for i, e in enumerate(df['target']):
    labelsOneHot[i][e]=1
labelsOneHot.shape

(7613, 2)

In [19]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [36]:
keras.utils.set_random_seed(1234)
ngram_NN = keras.Sequential()
ngram_NN.bias = 0.2
ngram_NN.add(keras.layers.Input(2363))

ngram_NN.add(keras.layers.Reshape((139, 17)))

ngram_NN.add(keras.layers.Convolution1D(128, (4), strides=(2), activation='relu'))#, padding="causal"))
ngram_NN.add(keras.layers.MaxPool1D(2))

ngram_NN.add(keras.layers.Convolution1D(64,  (4), strides=(2), activation='relu'))#, padding="causal"))
ngram_NN.add(keras.layers.MaxPool1D(2))

ngram_NN.add(keras.layers.Flatten())

ngram_NN.add(keras.layers.Dense(2363, activation='relu'))
ngram_NN.add(keras.layers.Dense(1024, activation='relu'))
ngram_NN.add(keras.layers.Dense(512, activation='relu'))
ngram_NN.add(keras.layers.Dense(128,  activation='relu'))
ngram_NN.add(keras.layers.Dense(32,  activation='relu'))
ngram_NN.add(keras.layers.Dense(8,  activation='relu'))



ngram_NN.add(keras.layers.Dropout(.5))


ngram_NN.add(keras.layers.Dense(2, activation='softmax'))
ngram_NN.output_shape

(None, 2)

In [21]:
callbacks = [
    keras.callbacks.ModelCheckpoint("hdt5_save_at_{epoch}.h5"),
]

In [37]:
ngram_NN.compile( metrics=["accuracy"], 
optimizer = keras.optimizers.Adam(learning_rate=1e-3), 
loss="categorical_crossentropy")

In [38]:
ngram_NN.fit(n_gram_onehot, labelsOneHot, epochs=40, callbacks=callbacks)

Epoch 1/40
238/238 [==============================] - 15s 59ms/step - loss: 0.6595 - accuracy: 0.5992
Epoch 2/40
238/238 [==============================] - 11s 47ms/step - loss: 0.6305 - accuracy: 0.6231
Epoch 3/40
238/238 [==============================] - 10s 41ms/step - loss: 0.6095 - accuracy: 0.6338
Epoch 4/40
238/238 [==============================] - 10s 43ms/step - loss: 0.5992 - accuracy: 0.6423
Epoch 5/40
238/238 [==============================] - 9s 39ms/step - loss: 0.5911 - accuracy: 0.6883
Epoch 6/40
238/238 [==============================] - 9s 40ms/step - loss: 0.5799 - accuracy: 0.6937
Epoch 7/40
238/238 [==============================] - 9s 39ms/step - loss: 0.5732 - accuracy: 0.7005
Epoch 8/40
238/238 [==============================] - 9s 39ms/step - loss: 0.5627 - accuracy: 0.7076
Epoch 9/40
238/238 [==============================] - 9s 39ms/step - loss: 0.5570 - accuracy: 0.7087
Epoch 10/40
238/238 [==============================] - 9s 40ms/step - loss: 0.5455 - ac